In [212]:
data_collection=["Dataset1.csv","Dataset2.csv","Dataset3.csv","Dataset4.csv"]

In [213]:
newList=[]
import pandas as pd
for data_ in data_collection:
    frame = pd.read_csv(data_,index_col=None, header=0)
    newList.append(frame)

# concatenating all dataset
df = pd.concat(newList, ignore_index=True)

In [214]:
df

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...,...,...,...
1581,LneaDw26bFu3RCmyrWyP9S6wh1h9dBv3X95g1HzKAb4,Dany PK,NaN,SUBSCRIBE TO MY CHANNEL X PLEASE!. SPARE,1
1582,LneaDw26bFsD65dtIvAEObWYIYnFTqQDKBek_Ypz3J8,SmexyFriedChicken,NaN,Check out my videos guy! :) Hope you guys had ...,1
1583,LneaDw26bFuvs-8oWkLpAFa6g3QHpWD8k7sbbMP3Bg8,The Guy That's Done Everything,NaN,3 yrs ago I had a health scare but thankfully ...,1
1584,z12hfp2wmyuqztkw504cgblyxtbsxjuzeow0k,Jesse Pinkman,2015-05-06T11:42:44.601000,Rihanna looks so beautiful with red hair ;)﻿,0


# Data Cleaning
To improve efficiency of our model, let's replace all link tags with a common word so that our algorithm treats all links as a single feature in our classification.

In [215]:
#dropping unnecessary attributes

df.drop(columns=["COMMENT_ID","AUTHOR","DATE"],inplace=True)

In [216]:
# viewing the conntent of 700th data

df["CONTENT"][700]

'<a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&amp;t=2m19s">2:19</a> best part\ufeff'

In [217]:
import html
df["CONTENT"]=df["CONTENT"].apply(html.unescape)
df["CONTENT"]=df["CONTENT"].str.replace("\ufeff","")

In [218]:
df["CONTENT"][700]

'<a href="http://www.youtube.com/watch?v=KQ6zr6kCPj8&t=2m19s">2:19</a> best part'

In [219]:
# replace all a-tags with a word "htmllink"

df["CONTENT"]=df["CONTENT"].str.replace("(<a.+>)","htmllink",regex=True)

Now lets inspect all other tags and remove them as they are not relevant to our analysis. 

In [220]:
df[df["CONTENT"].str.contains("<.+>")]["CONTENT"]

381                      <script>document.write('htmllink
702     Hey guys, I'm a human.<br /><br /><br />But I ...
708                                          Awsome<br />
728                             Super awesome video<br />
730     This Will Always Be My Favorite Song<br />But ...
                              ...                        
1406                    Hello. İ am from Azerbaijan<br />
1409                EMINEM<3 <br />the best rapper ever<3
1499    If you are a person that loves real music you ...
1546               Love your songs<br />Supper cool<br />
1566     Really good song .<br />you know love song song.
Name: CONTENT, Length: 65, dtype: object

In [221]:
df["CONTENT"]=df["CONTENT"].str.replace("<.+>","",regex=True)

In [222]:
df["CONTENT"]=df["CONTENT"].str.replace("\'","")

In [223]:
df["CONTENT"]=df["CONTENT"].str.lower()

In [224]:
df[df["CONTENT"].str.contains("\.com|watch\?")]

,CONTENT,CLASS
2,just for test i have to say murdev.com,1
4,watch?v=vtarggvgtwq check this out .,1
12,https://twitter.com/gbphotographygb,1
14,please like :d https://premium.easypromosapp.c...,1
17,http://www.ebay.com/itm/171183229277?sspagenam...,1
...,...,...
1448,everyone come and check out the new gta 5 game...,1
1476,check out these lyrics /watch?v=yuttx04oyqq,1
1521,hello to everyone! please check out my video: ...,1
1522,/watch?v=aimbwbfqbzg watch and subscrible,1


There are still few rows with links. Let's replace them too with "htmllink" and then remove all non-alpha numerical characters.

In [225]:
df["CONTENT"][1573]

'subscribe to my channel  /watch?v=nxk32i0hkds'

In [226]:
df["CONTENT"]=df["CONTENT"].str.replace(r"\S*\.com\S*|\S*watch\?\S*","htmllink",regex=True)

In [227]:
df["CONTENT"]=df["CONTENT"].str.replace("\W"," ",regex=True)

In [228]:
df["CONTENT"][1573]

'subscribe to my channel  htmllink'

In [229]:
df["CONTENT"][14]

'please like  d htmllink'

## Model Creation

In [230]:
# calculating relative frequencies of the unique values

df["CLASS"].value_counts(normalize=True)

1    0.52396
0    0.47604
Name: CLASS, dtype: float64

Now let's get the unique words and perform the required calculations

In [231]:
# creating a list to store all words

vocab_list=[]
for comment in df["CONTENT"]:
    for word in comment.split():
        vocab_list.append(word)

In [232]:
vocabulary=list(set(vocab_list))
len(vocabulary)

3363

In [233]:
# Create a column for each of the unique word in our vocabulary inorder to get the count of words 
for word in vocabulary:
    df[word]=0

In [234]:
df.head()

,CONTENT,CLASS,animal,hating,friends,supporters,annoys,luck,5287,htmllinkand,...,user,mri,close,room,gave,erection,danke,toilet,vip,completely
0,huh anyway check out this you tube channel ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hey guys check out my new channel and our firs...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,just for test i have to say htmllink,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,me shaking my sexy ass on my channel enjoy _,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,htmllink check this out,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
# looping through data frame and counting words 

for index,value in enumerate(df["CONTENT"]):
  for l in value.split():
    df[l][index]+=1

In [241]:
df.head()

,CONTENT,CLASS,animal,hating,friends,supporters,annoys,luck,5287,htmllinkand,...,user,mri,close,room,gave,erection,danke,toilet,vip,completely
0,huh anyway check out this you tube channel ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,hey guys check out my new channel and our firs...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,just for test i have to say htmllink,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,me shaking my sexy ass on my channel enjoy _,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,htmllink check this out,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [242]:
#Total number of words in each class

df.groupby("CLASS").sum().sum(axis=1)

CLASS
0     7417
1    15560
dtype: int64

In [243]:
# Assign variables to all values required in calculation

p_ham=0.47604
p_spam=0.52396
n_spam=df[df["CLASS"]==1].drop(columns=["CONTENT","CLASS"]).sum().sum()
n_ham=df[df["CLASS"]==0].drop(columns=["CONTENT","CLASS"]).sum().sum()
n_vocabulary=len(vocabulary)

In [244]:
# Slicing dataframe for each class

df_sspam=df[df["CLASS"]==1]
df_hham=df[df["CLASS"]==0]

In [245]:
#creating dictionaries to store probabilities of each word in dataframe given its a spam and not spam

parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}
for word in vocabulary:
    n_word_given_spam = df_sspam[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + 1) / (n_spam + 1*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    n_word_given_ham = df_hham[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + 1) / (n_ham + 1*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Model Testing

In [246]:
#defining a function to clean and classify all data

def classifier(string):
    message=html.unescape(string)
    message=string.replace("\ufeff","")
    message=string.replace("(<a.+>)","htmllink")
    message=string.replace("\'|<.+>","")
    message=string.replace("\S*\.com\S*|\S*watch\?\S*","htmllink")
    message=string.replace("\W"," ").lower()
    p_string_s=1
    p_string_h=1
    for word in message.split():
        if word in parameters_spam:
            p_string_s*=parameters_spam[word]
            p_string_h*=parameters_ham[word]
    if (p_string_s*p_spam)>(p_string_h*p_ham):
        return(1)
    elif (p_string_s*p_spam)<(p_string_h*p_ham):
        return(0)
    else:
        return(-1)

In [247]:
# Reading the dataframe for testing model
df_test=pd.read_csv("Dataset5.csv")

In [248]:
df_test.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,Nice song﻿,0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,I love song ﻿,0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,2015-05-28T21:00:08.607000,I love song ﻿,0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,"860,000,000 lets make it first female to reach...",0
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,2015-05-28T17:08:29.827000,shakira is best for worldcup﻿,0


In [249]:
#classifying the test dataset

df_test["Pred_Class"]=df_test["CONTENT"].apply(classifier)

In [250]:
#calculating accuracy of the model

correct_predictions=0
total_rows=0
for row in df_test.iterrows():
    row=row[1]
    total_rows+=1
    if row["CLASS"]==row["Pred_Class"]:
        correct_predictions+=1
accuracy=(correct_predictions/total_rows)*100
print("accuracy=",accuracy)


accuracy= 84.32432432432432


In [204]:
classifier("I like his channel very much")

1

In [205]:
classifier("I like him very much")

0

In [206]:
classifier("My content is good guys. check out")

1

In [207]:
classifier("My content is good")

0

In [208]:
classifier("My laptop is good")

0

In [209]:
classifier("My content is better than his")

1

In [210]:
classifier("Roses are red")

0

In [211]:
classifier("Excellent video!")

1